In [1]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

/home/redix/hansol/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

/home/redix/hansol/.venv/lib/python3.10/site-packages/sentence_transformers/SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
sample = pd.read_csv("sample_submission.csv")

In [4]:
grouped = train.groupby("인적사고")

In [5]:
res = {}
cosine_res = []
for name, group in tqdm(grouped):
    plan = group["재발방지대책 및 향후조치계획"]
    vectors = np.stack(plan.apply(model.encode).to_numpy())
    similarity = cosine_similarity(vectors, vectors)    
    cosine_res += similarity[similarity.mean(axis=1).argmax()].tolist()
    res[name] = plan.iloc[similarity.mean(axis=1).argmax()]

  0%|          | 0/23 [00:00<?, ?it/s]

100%|██████████| 23/23 [03:10<00:00,  8.30s/it]


In [6]:
arr = cosine_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")


Range 0.0 - 0.1: 1개
Range 0.1 - 0.2: 25개
Range 0.2 - 0.3: 191개
Range 0.3 - 0.4: 665개
Range 0.4 - 0.5: 1657개
Range 0.5 - 0.6: 3794개
Range 0.6 - 0.7: 6597개
Range 0.7 - 0.8: 7557개
Range 0.8 - 0.9: 2783개
Range 0.9 - 1.0: 113개


In [10]:
res

{'감전': '작업자 교육 및 안전장구 착용 철저와 전기작업자 교육 및 피복관리 철저를 통한 재발 방지 대책.',
 '교통사고': '작업업 특별 안전 교육 실시, 신호수 교육 실시, 작업차량 후방 감지 센서 및 카메라 설치, 수시 위험성 평가 실시, 감독관 주체 특별 안전 교육 실시, 현장 점검 시 신호수 위치 점검 등으로 구성된 재발 방지 대책 및 향후 조치 계획.',
 '기타': '작업전 안전교육 및 특별안전교육 실시와 안전관리 및 안전교육 철저를 통한 재발 방지 대책.',
 '깔림': '작업 개시 전 작업내용 숙지 및 안전교육 강화, 수시 현장 방문 및 점검을 통한 재발 방지 대책 강구.',
 '끼임': '작업전 작업자 안전교육 강화, 안전수칙 준수 특별안전교육 실시, 작업지휘자 관리감독 철저에 대한 재발 방지 대책과 향후 조치 계획.',
 '넘어짐(기타)': '작업전 안전교육 실시와 안전관리자 안전점검 실시를 통한 재발 방지 대책 및 향후 조치 계획.',
 '넘어짐(물체에 걸림)': '작업전 안전교육 강화 및 작업장 위험요소 점검을 통한 재발 방지와 안전관리 교육 철저를 통한 향후 조치 계획.',
 '넘어짐(미끄러짐)': '이동통로 확보 관리와 작업 전 안전교육 철저 및 정기적 근로자 안전교육 시행을 통한 재발 방지 대책.',
 '떨어짐(10미터 이상)': '안전시설물 설치와 체계적인 안전교육 실시를 통한 재발 방지 대책 및 공사현장 작업중지명령과 안전교육 강화를 포함한 향후 조치 계획.',
 '떨어짐(2미터 미만)': '안전교육 실시와 현장 내 작업지시사항 철저 이행, 안전관리 교육 이행, 안전위험요소 제거 점검 및 대책 강구를 통한 재발 방지 대책.',
 '떨어짐(2미터 이상 ~ 3미터 미만)': '작업 전 안전교육 철저와 안전고리 이중결속 준수, 작업 중 안전시설물 확인 및 사고자 상태 수시 체크를 통한 재발 방지 대책 마련.',
 '떨어짐(3미터 이상 ~ 5미터 미만)': '작업 시 안전교육 및 보호구 착용과 안전매트 설치를 통한 재발 방지 대책

In [12]:
res_v = {}
for k,v in res.items():
    res_v[k] = model.encode(v)

In [15]:
sample.head(3)

,ID,재발방지대책 및 향후조치계획,vec_0,vec_1,vec_2,vec_3,vec_4,vec_5,vec_6,vec_7,...,vec_758,vec_759,vec_760,vec_761,vec_762,vec_763,vec_764,vec_765,vec_766,vec_767
0,TEST_000,안전교육 실시,-0.620284,-0.377644,-0.321682,0.148727,-0.193202,0.662165,1.092194,-0.043897,...,1.336321,0.882164,1.780816,2.430824,0.801544,0.016859,-0.152427,0.243186,0.267703,0.109774
1,TEST_001,안전교육 실시,-0.620284,-0.377644,-0.321682,0.148727,-0.193202,0.662165,1.092194,-0.043897,...,1.336321,0.882164,1.780816,2.430824,0.801544,0.016859,-0.152427,0.243186,0.267703,0.109774
2,TEST_002,안전교육 실시,-0.620284,-0.377644,-0.321682,0.148727,-0.193202,0.662165,1.092194,-0.043897,...,1.336321,0.882164,1.780816,2.430824,0.801544,0.016859,-0.152427,0.243186,0.267703,0.109774


In [21]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Columns: 770 entries, ID to vec_767
dtypes: float64(768), object(2)
memory usage: 5.7+ MB


In [22]:
for i in range(len(test)):
    accident = test.loc[i, "인적사고"]
    sample.loc[i, "재발방지대책 및 향후조치계획"] = res[accident]
    sample.iloc[i, 2:] = res_v[accident]

In [24]:
sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Columns: 770 entries, ID to vec_767
dtypes: float64(768), object(2)
memory usage: 5.7+ MB


In [25]:
sample.to_csv("baseline.csv", index=False, encoding='utf-8-sig')